## GOAL OF PROJECT

To implement the following paper "Predicting the direction of stock market prices using random forest" - Luckyson Khaidem, Snehanshu Saha, Sudeepa Roy Dey

### IMPORTING THE LIBRARIES REQUIRED FOR THE TASK 

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

## The model of choice for training and prediction purposes is the Random Forest Classifier

### Importing required libraries for training and testing the model

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, confusion_matrix, recall_score, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import TimeSeriesSplit

### In order to use the technical financial indicators required in the paper, we utilise a pre-defined library as follows:

## https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py

#### Importing the above library

In [3]:
import pandas_techinal_indicators as ta 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Data

We are using data from a variety of pharma companies to form trend for the market 

In [4]:
#we are trying to predict the movements of CIPLA given the data for other related stocks

raw_data = pd.read_csv('CIPLA.csv')

del(raw_data['Series'])
del(raw_data['Date'])
del(raw_data['Symbol'])
del(raw_data['N'])

In [5]:
raw_data2 = pd.read_csv('CADILAHC.csv')

del(raw_data2['Series'])
del(raw_data2['Date'])
del(raw_data2['Symbol'])
del(raw_data2['N'])

In [6]:
raw_data3 = pd.read_csv('DRREDDY.csv')

del(raw_data3['Series'])
del(raw_data3['Date'])
del(raw_data3['Symbol'])
del(raw_data3['N'])

In [7]:
raw_data4 = pd.read_csv('LUPIN.csv')

del(raw_data4['Series'])
del(raw_data4['Date'])
del(raw_data4['Symbol'])
del(raw_data4['N'])

In [8]:
raw_data5 = pd.read_csv('SUNPHARMA.csv')

del(raw_data5['Series'])
del(raw_data5['Date'])
del(raw_data5['Symbol'])
del(raw_data5['N'])

## Exponential Smoothing 

As indicated by the authors of the paper, this is done for the purpose of putting more importance on recent data and exponentially decreasing weightage to past data 

In [9]:
# Function for exponentially smoothing

def exp_smoothing(df, alpha):
    es_data = df.ewm(alpha=alpha).mean()    
    return es_data

In [10]:
# For current testing purposes, value of alpha used is 0.9

sdata = exp_smoothing(raw_data, 0.9)
sdata2 = exp_smoothing(raw_data2, 0.9)
sdata3 = exp_smoothing(raw_data3, 0.9)
sdata4 = exp_smoothing(raw_data4, 0.9)
sdata5 = exp_smoothing(raw_data5, 0.9)
# Let us visualise the data

sdata.head() 

,Prev Close,Open,High,Low,Last,Close,Average,Volume,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,626.400000,626.500000,634.800000,626.500000,629.950000,628.400000,630.720000,5.967870e+05,3.764074e+08,9333.000000,201185.000000,33.710000
1,628.218182,632.409091,637.300000,629.090909,630.904545,629.990909,632.756364,6.806161e+05,4.306800e+08,10921.181818,320182.272727,46.882727
2,629.958559,626.635135,637.930631,626.306306,632.251802,632.701802,633.417297,1.223210e+06,7.748409e+08,21567.864865,652569.414414,53.042432
3,632.696130,630.563906,632.592529,612.079208,615.778533,616.183528,621.124500,1.576297e+06,9.785972e+08,53017.131413,956747.259226,60.483987
4,616.184448,615.971245,619.279120,605.257853,616.067856,612.373315,612.462363,8.901653e+05,5.458053e+08,36130.144271,429642.354874,47.079265


## Technical indicators used for Feature Extraction


In [11]:
def feature_extraction(data):
    for x in [5, 14, 26, 44, 66]:
        data = ta.relative_strength_index(data, n=x)
        data = ta.stochastic_oscillator_d(data, n=x)
        data = ta.accumulation_distribution(data, n=x)
        data = ta.average_true_range(data, n=x)
        data = ta.momentum(data, n=x)
        data = ta.money_flow_index(data, n=x)
        data = ta.rate_of_change(data, n=x)
        data = ta.on_balance_volume(data, n=x)
        data = ta.commodity_channel_index(data, n=x)
        data = ta.ease_of_movement(data, n=x)
        data = ta.trix(data, n=x)
        data = ta.vortex_indicator(data, n=x)
        data = ta.moving_average(data, n=x)
        data = ta.standard_deviation(data, n=x) 
        data = ta.keltner_channel(data, n=x)
        data = ta.coppock_curve(data, n=x)
        data = ta.force_index(data, n=x)
        data = ta.bollinger_bands(data, n=x)
        data = ta.exponential_moving_average(data, n=x)
    
    data = ta.ppsr(data)
    data = ta.stochastic_oscillator_k(data)
    data = ta.mass_index(data)
    data = ta.ultimate_oscillator(data)
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema14'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()
    data = ta.chaikin_oscillator(data)    
         
    data = ta.macd(data, n_fast=12, n_slow=26)
    
    del(data['Open'])
    del(data['Prev Close'])
    del(data['High'])
    del(data['Low'])
    del(data['Volume'])
    del(data['Last'])
    del(data['Average'])
    del(data['Turnover'])
    del(data['No. of Trades'])
    del(data['Deliverable Qty'])
    del(data['% Dly Qt to Traded Qty'])
    
    return data
   
def compute_prediction_int(df, n):
    pred = (df.shift(-n)['Close'] >= df['Close'])
    pred = pred.iloc[:-n]
    return pred.astype(int)

def prepare_data(df, horizon):
    data = feature_extraction(df).dropna().iloc[:-horizon]
    data['pred'] = compute_prediction_int(data, n=horizon)
    del(data['Close'])
    return data.dropna()

## Preparation of training data and labels.

### Assume a prediction horizon of 1 days 

In [12]:
data = prepare_data(sdata, 1)
print('data done')
data2 = prepare_data(sdata2, 1)
print('data2 done')
data3 = prepare_data(sdata3, 1)
print('data3 done')
data4 = prepare_data(sdata4, 1)
print('data4 done')
data5 = prepare_data(sdata5, 1)
print('data5 done')


## Identifying and extracting the label
data2 = data2.add_suffix('_2')
data3 = data3.add_suffix('_3')
data4 = data4.add_suffix('_4')
data5 = data5.add_suffix('_5')

print('suffixes added')


data = pd.concat([data, data2], axis=1)
data = pd.concat([data, data3], axis=1)
data = pd.concat([data, data4], axis=1)
data = pd.concat([data, data5], axis=1)

## Extracting the input features and creating the input feature matrix

data = data.dropna()
del(data['pred_2'])
del(data['pred_3'])
del(data['pred_4'])
del(data['pred_5'])

input_feature = [x for x in data.columns if x not in ['gain', 'pred']]
X = data[input_feature]
y = data['pred']

#print("This is X: ")
#print(X)
#print("This is y: ")
#print(y)

data done
data2 done
data3 done
data4 done
data5 done
suffixes added


### Scaling the data into range (-1,1) for pre-processing  

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_new = scaler.fit_transform(X)

#print("This is X_new: ")
#print(X_new)

## HERE, instead of splitting into training and testing radnomly (as is done usually), we are using the TimeSeriesSplit method to split the data into training and testing data

### This is because, since we are using time series data, random splitting may lead to data leakage

In [14]:
tscv = TimeSeriesSplit()

for train_index, test_index in tscv.split(X_new):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_new[train_index, :], X_new[test_index,:]
    
for train_index, test_index in tscv.split(y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

## Printing sizes to verify correctness
print('len X_train', len(X_train))
print('len y_train', len(y_train))
print('len X_test', len(X_test))
print('len y_test', len(y_test))

len X_train 946
len y_train 946
len X_test 189
len y_test 189


## MODEL - Random Forests 

In [15]:
model = RandomForestClassifier(n_jobs=-1, n_estimators=111, random_state=42)

## @EVERYONE - Try playing with the parameters above - increasing number of estimators may help improving accuracy

## Training and testing the model

In [16]:
model.fit(X_train, y_train.values.ravel());
print("model fitted")
prediction = model.predict(X_test)
print("prediction created")
print(np.size(prediction))
print(np.size(y_test))

accuracy = accuracy_score(y_pred=prediction, y_true=y_test)

print('Accuracy: {0:1.2f}'.format(accuracy))


confusion = confusion_matrix(y_pred=prediction, y_true=y_test)
print('Confusion Matrix')
print(confusion)


precision = precision_score(y_pred=prediction, y_true=y_test)
recall = recall_score(y_pred=prediction, y_true=y_test)
f1 = f1_score(y_pred=prediction, y_true=y_test)
print('Precision: {0:1.2f}, Recall: {1:1.2f}, f1: {2:1.2f}'.format(precision, recall, f1))


model fitted
prediction created
189
189
Accuracy: 0.54
Confusion Matrix
[[63 40]
 [47 39]]
Precision: 0.49, Recall: 0.45, f1: 0.47


## TRYING SVM IMPLEMENTATION 

In [17]:
from sklearn import svm
# model = svm.SVC(gamma = 1 / (X_train.shape[-1] * X_train.var()))

model = svm.SVC()

model.fit(X_train, y_train.values.ravel());
print("model fitted")
prediction = model.predict(X_test)
print("prediction created")
print(np.size(prediction))
print(np.size(y_test))

accuracy = accuracy_score(y_pred=prediction, y_true=y_test)

print('Accuracy: {0:1.2f}'.format(accuracy))


confusion = confusion_matrix(y_pred=prediction, y_true=y_test)
print('Confusion Matrix')
print(confusion)


precision = precision_score(y_pred=prediction, y_true=y_test)
recall = recall_score(y_pred=prediction, y_true=y_test)
f1 = f1_score(y_pred=prediction, y_true=y_test)
print('Precision: {0:1.2f}, Recall: {1:1.2f}, f1: {2:1.2f}'.format(precision, recall, f1))

model fitted
prediction created
189
189
Accuracy: 0.58
Confusion Matrix
[[96  7]
 [72 14]]
Precision: 0.67, Recall: 0.16, f1: 0.26
